<a href="https://colab.research.google.com/github/V1A/Prueba/blob/main/TEM1_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Lineal

El dataset que emplearemos ahora tiene como objetivo determinar la potencia de salida de una planta eléctrica. Los features son los siguientes:

- Temperature (AT) in the range 1.81°C and 37.11°C,
- Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
- Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%

La respuesta está data en la columna PE, de las siguientes características:

- Net hourly electrical energy output (PE) 420.26-495.76 MW

In [81]:
#Obtener el dataset a partir de:
!wget http://www.innovacademy.com/my_resources/retail_db.zip

--2021-11-10 19:45:15--  http://www.innovacademy.com/my_resources/retail_db.zip
Resolving www.innovacademy.com (www.innovacademy.com)... 108.167.158.160
Connecting to www.innovacademy.com (www.innovacademy.com)|108.167.158.160|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1709365 (1.6M) [application/zip]
Saving to: ‘retail_db.zip.2’

retail_db.zip.2     100%[===================>]   1.63M  5.75MB/s    in 0.3s    

2021-11-10 19:45:15 (5.75 MB/s) - ‘retail_db.zip.2’ saved [1709365/1709365]



In [82]:
!unzip retail_db.zip

Archive:  retail_db.zip
replace retail_db/categories/part-00000? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [83]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [84]:
!pip install -q py4j
!pip install pyspark
!pip install joblibspark

In [85]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sb

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

from pyspark.sql.types import *
import pyspark.sql.functions as F

Inicializar la sesión de spark a través de getOrCreate()

In [86]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [87]:
!ls -l retail_db

total 24
drwxr-xr-x 2 root root 4096 Sep 14  2019 categories
drwxr-xr-x 2 root root 4096 Sep 14  2019 customers
drwxr-xr-x 2 root root 4096 Sep 14  2019 departments
drwxr-xr-x 2 root root 4096 Sep 14  2019 order_items
drwxr-xr-x 2 root root 4096 Sep 14  2019 orders
drwxr-xr-x 2 root root 4096 Sep 14  2019 products


Definir el esquema que refleje los tipos de datos de cada columna (Con StructType y StructField)

In [88]:
#Crear los esquemas correspondientes a cada tabla del dataset
departments_schema = StructType([
    StructField('department_id', IntegerType(), False),
    StructField('department_name', StringType(), False)
])

categories_schema = StructType([
    StructField('category_id', IntegerType(), False),
    StructField('category_department_id', IntegerType(), False),
    StructField('category_name', StringType(), False)
])

products_schema = StructType([
    StructField('product_id', IntegerType(), False),
    StructField('product_category_id', IntegerType(), False),
    StructField('product_name', StringType(), False),
    StructField('product_description', StringType(), False),
    StructField('product_price', FloatType(), False),
    StructField('product_image', StringType(), False)
])

orders_schema = StructType([
    StructField('order_id', IntegerType(), False),
    StructField('order_date', DateType(), False),
    StructField('order_customer_id', IntegerType(), False),
    StructField('order_status', StringType(), False)
])

order_items_schema = StructType([
    StructField('order_item_id', IntegerType(), False),
    StructField('order_item_order_id', IntegerType(), False),
    StructField('order_item_product_id', IntegerType(), False),
    StructField('order_item_quantity', IntegerType(), False),
    StructField('order_item_subtotal', FloatType(), False),
    StructField('order_item_product_price', IntegerType(), False)
])

customers_schema = StructType([
    StructField('customer_id', IntegerType(), False),
    StructField('customer_fname', StringType(), False),
    StructField('customer_lname', StringType(), False),
    StructField('customer_email', StringType(), False),
    StructField('customer_password', StringType(), False),
    StructField('customer_street', StringType(), False),
    StructField('customer_city', StringType(), False),
    StructField('customer_state', StringType(), False),
    StructField('customer_zipcode', StringType(), False)
])

customers = spark.read.schema(customers_schema).csv('retail_db/customers')
orders = spark.read.schema(orders_schema).csv('retail_db/orders')
products = spark.read.schema(products_schema).csv('retail_db/products')
order_items = spark.read.schema(order_items_schema).csv('retail_db/order_items')

In [89]:
order_items.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                    null|
|            2|                  2|                 1073|                  1|             199.99|                    null|
|            3|                  2|                  502|                  5|              250.0|                    null|
|            4|                  2|                  403|                  1|             129.99|                    null|
|            5|                  4|                  897|                  2|              49.98|                    null|
|            6| 

In [90]:
#Leer los dataframes para cada archivo
departments = spark.read.schema(departments_schema).csv('retail_db/departments')
departments.show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            2|        Fitness|
|            3|       Footwear|
|            4|        Apparel|
|            5|           Golf|
|            6|       Outdoors|
|            7|       Fan Shop|
+-------------+---------------+



In [91]:
categories = spark.read.schema(categories_schema).csv('retail_db/categories')
categories.show()

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
|          6|                     2|   Tennis & Racquet|
|          7|                     2|             Hockey|
|          8|                     2|        More Sports|
|          9|                     3|   Cardio Equipment|
|         10|                     3|  Strength Training|
|         11|                     3|Fitness Accessories|
|         12|                     3|       Boxing & MMA|
|         13|                     3|        Electronics|
|         14|                     3|     Yoga & Pilates|
|         15|                  

In [92]:
products = spark.read.schema(products_schema).csv('retail_db/products')
products.show()

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|       134.99|http://images.acm...|
|         7|       

In [93]:
orders = spark.read.schema(orders_schema).csv('retail_db/orders')
orders.show()

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|2013-07-25|            11599|         CLOSED|
|       2|2013-07-25|              256|PENDING_PAYMENT|
|       3|2013-07-25|            12111|       COMPLETE|
|       4|2013-07-25|             8827|         CLOSED|
|       5|2013-07-25|            11318|       COMPLETE|
|       6|2013-07-25|             7130|       COMPLETE|
|       7|2013-07-25|             4530|       COMPLETE|
|       8|2013-07-25|             2911|     PROCESSING|
|       9|2013-07-25|             5657|PENDING_PAYMENT|
|      10|2013-07-25|             5648|PENDING_PAYMENT|
|      11|2013-07-25|              918| PAYMENT_REVIEW|
|      12|2013-07-25|             1837|         CLOSED|
|      13|2013-07-25|             9149|PENDING_PAYMENT|
|      14|2013-07-25|             9842|     PROCESSING|
|      15|2013-07-25|             2568|       CO

In [94]:
order_items = spark.read.schema(order_items_schema).csv('retail_db/order_items')
order_items.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                    null|
|            2|                  2|                 1073|                  1|             199.99|                    null|
|            3|                  2|                  502|                  5|              250.0|                    null|
|            4|                  2|                  403|                  1|             129.99|                    null|
|            5|                  4|                  897|                  2|              49.98|                    null|
|            6| 

In [95]:
customers = spark.read.schema(customers_schema).csv('retail_db/customers')
customers.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [99]:
#Obtener los clientes (customers) cuyo nombre empieza con “J” o
#cuyo apellido termina con “h”
customers = customers.where(F.col('customer_fname').like('%J%'))
customers.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|         15|          Jane|          Luna|     XXXXXXXXX|        XXXXXXXXX|    673 Burning Glen|      Fontana|            CA|           92336|
|         22|        Joseph|         Smith|     XXXXXXXXX|        XXXXXXXXX|7740 Broad Fox Vi...| North Bergen|            NJ|           07047|
|         26|        Johnny|          Hood|     XXXXXXXXX|        XXXXXXXXX|9576 Middle Hills...|     Glenview|            IL|           60025|
|         39|          Juan|      Mckinney|     XXXXXXXXX|        XXXXXXXXX|7274 Blue Wagon  ...|       Caguas|            PR|          

In [101]:
#Obtener los departamentos conjuntamente con sus categorias
departments_categories = departments.join(categories, departments['department_id'] == categories.category_department_id, how='left_outer')
departments_categories.show()

+-------------+---------------+-----------+----------------------+-------------------+
|department_id|department_name|category_id|category_department_id|      category_name|
+-------------+---------------+-----------+----------------------+-------------------+
|            2|        Fitness|          8|                     2|        More Sports|
|            2|        Fitness|          7|                     2|             Hockey|
|            2|        Fitness|          6|                     2|   Tennis & Racquet|
|            2|        Fitness|          5|                     2|           Lacrosse|
|            2|        Fitness|          4|                     2|         Basketball|
|            2|        Fitness|          3|                     2|Baseball & Softball|
|            2|        Fitness|          2|                     2|             Soccer|
|            2|        Fitness|          1|                     2|           Football|
|            3|       Footwear|         16|

In [102]:
#Obtener los 5 primeros productos con los precios mas altos 
